# Laboratorio 2 Data Science
- Kenneth Gálvez 20079
- José Mariano Reyes 20074

## 1. Preparación de datos

In [2]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Cargar los datos desde los archivos .p
with open('entrenamiento.p', 'rb') as f:
    train_data = pickle.load(f)
    
with open('validacion.p', 'rb') as f:
    validation_data = pickle.load(f)
    
with open('prueba.p', 'rb') as f:
    test_data = pickle.load(f)

# Dividir los datos en imágenes y etiquetas
x_train, y_train = train_data['features'], train_data['labels']
x_val, y_val = validation_data['features'], validation_data['labels']
x_test, y_test = test_data['features'], test_data['labels']

# Preprocesamiento: Redimensionar y Normalizar
input_shape = (32, 32, 3)  # Por ejemplo, redimensionar a 32x32 y 3 canales de color (RGB)
x_train = x_train / 255.0  # Normalización
x_val = x_val / 255.0
x_test = x_test / 255.0

# Preprocesamiento: Codificación One-Hot para las etiquetas
num_classes = 43
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)


## 2. Implementación de la arquitectura Le-Net

### Arquitectura LeNet:

La arquitectura LeNet fue propuesta por Yann LeCun en 1998 y es considerada una de las primeras arquitecturas exitosas de redes neuronales convolucionales. Fue diseñada para el reconocimiento de caracteres escritos a mano y, aunque es relativamente simple en comparación con las CNN modernas, sienta las bases para muchas de las arquitecturas posteriores. La arquitectura LeNet consta de las siguientes capas:

1. Capa de Convolución (C1): Convolución 2D con filtros (kernels) de tamaño pequeño. Aplicación de función de activación, típicamente ReLU (Rectified Linear Unit).Captura características locales en la imagen, como bordes y texturas.

2. Capa de Sub-muestreo (Pooling) (S2): Operación de sub-muestreo, como MaxPooling, para reducir el tamaño de la imagen. Ayuda a conservar las características más importantes y reduce la cantidad de parámetros.

3. Capa de Convolución (C3): Otra capa de convolución con filtros más grandes. Función de activación ReLU. Captura características más abstractas basadas en las características capturadas en C1.

4. Capa de Sub-muestreo (Pooling) (S4): Otra operación de sub-muestreo para reducir el tamaño aún más.

5. Capa completamente conectada (Fully Connected) (F5): Capa densamente conectada con unidades de activación ReLU. Captura relaciones más complejas entre las características capturadas en capas anteriores.

6. Capa completamente conectada (Fully Connected) (F6): Capa final que produce las salidas finales de clasificación. Puede tener unidades de activación lineales o softmax, dependiendo del problema.

### Diagrama de la Red LeNet (png entregado aparte)

### Proceso de Convolución, Función de Activación y Pooling:

1. Convolución: En la capa de convolución, los filtros se deslizan sobre la imagen de entrada realizando productos escalares locales. Esto ayuda a detectar patrones específicos como bordes, texturas y características relevantes en la imagen.

2. Función de Activación: Después de cada operación de convolución, se aplica una función de activación, como la función ReLU. ReLU convierte los valores negativos en cero y mantiene los valores positivos sin cambios. Esto introduce no linealidades en la red, lo que le permite capturar relaciones más complejas.

3. Pooling (Sub-muestreo): La operación de pooling reduce el tamaño espacial de la representación de la imagen mientras mantiene las características más importantes. MaxPooling, por ejemplo, selecciona el valor máximo dentro de una ventana en la imagen. Esto ayuda a reducir la cantidad de parámetros y hacer que la red sea más robusta a pequeñas transformaciones en la imagen.

## 3. Construcción del modelo

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Definición del modelo LeNet
model = models.Sequential()

# Capa de Convolución (C1)
model.add(layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 3)))
# Capa de Sub-muestreo (S2)
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Capa de Convolución (C3)
model.add(layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
# Capa de Sub-muestreo (S4)
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Aplanar los datos para las capas Fully Connected
model.add(layers.Flatten())

# Capa Fully Connected (F5)
model.add(layers.Dense(units=120, activation='relu'))

# Capa Fully Connected (F6)
model.add(layers.Dense(units=84, activation='relu'))

# Capa de Salida
model.add(layers.Dense(units=43, activation='softmax'))  # 43 clases en este caso

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
